# Session 4 - Word replacement, word analysis

### Real world application of text mining
Paper published by Dr. San Cannon: https://www.kansascityfed.org/~/media/files/publicat/econrev/econrevarchive/2015/4q15cannon.pdf


### Other people do this stuff too

#### Word counts:  https://www.federalreserve.gov/econres/notes/feds-notes/the-fomc-meeting-minutes-an-update-of-counting-words-20170803.htm


### Stemmed word clouds:
http://eprints.lse.ac.uk/60287/1/dp1276.pdf

# Alright... let's get started

In [19]:
#annoying bit about python: lack of native floating point division
from __future__ import division

In [20]:
# import module(s) into namespace
import pandas as pd
import numpy as np 
pathname = "C:\\Users\\Paul\\Desktop\\Rockhurst\\BIA 6304-Text Mining\\Week 2\\"
pd.set_option('display.max_colwidth', 15000) #important for getting all the text


### Recap: News example

We're going to read in our news articles, instantiate a vectorizer, and look at the top features


In [21]:
newsdf = pd.read_csv(pathname + "nytimes2013.csv", index_col = 0) 
print(newsdf.shape)
print(list(newsdf)) #equivalent of showing the headers of a data frame
print(type(newsdf))

(3848, 5)
['date', 'description', 'headline', 'url', 'text']
<class 'pandas.core.frame.DataFrame'>


In [22]:
newsdf.head()

date  \
0  2013-01-01   
1  2013-01-01   
2  2013-01-01   
3  2013-01-01   
4  2013-01-01   

                                                                                                                                                                           description  \
0             Ending a climactic showdown in the final hours of the 112th Congress, the House sent to President Obama legislation to avert big income tax increases on most Americans.   
1                        A report on nearly three million people found that those whose body mass index ranked them as overweight had less risk of dying than people of normal weight.   
2  As the United States prepares to withdraw from an unpopular war in Afghanistan, it faces challenges similar to what the country’s last occupier, the Soviet Union, had experienced.   
3            The popularity of the drinks reflects success in convincing consumers that they provide an edge, but most of their ingredients have no or little benefit, research shows.   
4                                 New Hampshire, which again chose a woman to be governor, will also become the first state in history to have an all-female delegation in Washington.   

                                                                 headline  \
0          Divided House Passes Tax Deal in End to Latest Fiscal Standoff   
1  Study Suggests Lower Mortality Risk for People Deemed to Be Overweight   
2                      With U.S. Set to Leave Afghanistan, Echoes of 1989   
3              Energy Drinks Promise Edge, but Experts Say Proof Is Scant   
4           From Congress to Halls of State, in New Hampshire, Women Rule   

                                                                                                              url  \
0                                  http://www.nytimes.com/2013/01/02/us/politics/house-takes-on-fiscal-cliff.html   
1                http://www.nytimes.com/2013/01/02/health/study-suggests-lower-death-risk-for-the-overweight.html   
2         http://www.nytimes.com/2013/01/02/world/asia/us-war-in-afghanistan-has-echoes-of-soviet-experience.html   
3           http://www.nytimes.com/2013/01/02/health/scant-proof-is-found-to-back-up-claims-by-energy-drinks.html   
4  http://www.nytimes.com/2013/01/02/us/politics/from-congress-to-halls-of-state-in-new-hampshire-women-rule.html   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [23]:
from nltk.corpus import stopwords

#create an object storing the default nltk stopwords
nltk_stopwords = stopwords.words("english") 

#create a custom stop words list by adding a few terms to the nltk list
my_stopwords = nltk_stopwords + ["000", "mr", "said", "says","say", "ms"]

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=False, stop_words = my_stopwords, min_df=.10, max_df =.60) 
cv_dm = cv.fit_transform(newsdf['text'])
print(cv_dm.shape)
names = cv.get_feature_names()   #create list of feature names
count = np.sum(cv_dm.toarray(), axis = 0) # add up feature counts 
count2 = count.tolist()  # convert numpy array to list
count_df = pd.DataFrame(count2, index = names, columns = ['count']) # create a dataframe from the list
count_df.sort_values(['count'], ascending = False)[0:19]  #arrange by count instead of alphabetical (top 20)

(3848, 495)


,count
people,5547
could,4620
last,4247
many,4211
first,4057
even,4029
city,3514
states,3247
state,3139
president,2991


In [25]:
#lets try some basic stemming - this will take a minute or ten

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer() 

def stem_text(row):
    text = str(row).split() #splits the text apart before stemming
    stemtext = [ps.stem(word) for word in text] #tells it which stemmer to apply and how
    stem2text = ' '.join(stemtext) #puts everything back together again
    return stem2text

newsdf['stemmed'] = newsdf["text"].apply(lambda x: stem_text(x)) #apply the above function to our text
#print( newsdf.text[0:1])
#print("~~~~~~~~~~~~~~~~~~~")
#print(newsdf.stemmed[0:1])

In [26]:
#how does that affect the feature list

cv_dm = cv.fit_transform(newsdf['stemmed'])
print(cv_dm.shape)
names = cv.get_feature_names()   
count = np.sum(cv_dm.toarray(), axis = 0) 
count2 = count.tolist()  
count_df = pd.DataFrame(count2, index = names, columns = ['count']) 
count_df.sort_values(['count'], ascending = False)[0:19]  

(3848, 592)


,count
state,5530
peopl,4698
could,4620
last,4384
mani,4141
work,4094
first,4057
use,3934
go,3909
get,3720


#### Is this a "better" feature space?  Why?  Why not?  How is it different from the feature space of the un-stemmed text?  Why?

## Custom Replacement

#### What if stemming is too much change? What if you only want to do some replacements and not others?

### How to create customized dictionaries for replacing terms

In [27]:
#simple string manipulation
word = 'cat'
print("old word: " + word)
word = word.replace('a','b')
print("new word: " + word)

old word: cat
new word: cbt


In [28]:
# can be more than one character at a time
sentence = "Bank of America reported lower earnings in Q4"
print("Old sentence: " + sentence)
sentence = sentence.replace("Bank of America","BofA")
print("New sentence: " + sentence)

Old sentence: Bank of America reported lower earnings in Q4
New sentence: BofA reported lower earnings in Q4


In [29]:
# using dictionaries

boat_dict = { 'ship':'boat', 'vessel':'boat'}
print(type(boat_dict))

<class 'dict'>


In [30]:
sampletext = 'what is the name of this vessel?'
print("Original text: " + sampletext)

for origword, newword in boat_dict.items():
    sampletext = sampletext.replace(origword.lower(), newword)
    
print("New text: " + sampletext)

Original text: what is the name of this vessel?
New text: what is the name of this boat?


In [31]:
#you can apply the change to an entire list

sampletext = ['Can you ship this to Cleveland?', 'What is the name of this battleship?']
newtext = []

print("Original text:" + str(sampletext))
for item in sampletext:
    for origword, newword in boat_dict.items():
        item = item.replace(origword.lower(), newword)
    newtext = newtext + [item]
    
print("New text: " + str(newtext))

Original text:['Can you ship this to Cleveland?', 'What is the name of this battleship?']
New text: ['Can you boat this to Cleveland?', 'What is the name of this battleboat?']


In [32]:
#you can apply the change to a column of a data frame
sampletext = {'text':['Can you ship this to Cleveland?', 'What is the name of this vessel?']}

sample_df = pd.DataFrame(sampletext, columns = ['text']) #create data frame from dictionary
print(sample_df)

print("~~~~~")

for origword, newword in boat_dict.items():
    sample_df.text = sample_df.text.str.replace(origword.lower(), newword)
    
print(sample_df)

                               text
0   Can you ship this to Cleveland?
1  What is the name of this vessel?
~~~~~
                              text
0  Can you boat this to Cleveland?
1   What is the name of this boat?


In [33]:
# more examples
address = "123 north anywhere street"

for word, initial in {"NORTH":"N", "SOUTH":"S" }.items():
    address = address.replace(word.lower(), initial)
print(address)

123 N anywhere street


### Misspelling suggestions
http://emnlp2014.org/papers/pdf/EMNLP2014171.pdf

### Build a custom dictionary to use as we did here
* Compare words in a corpus to a fairly comprehensive list of english words (such as Word Net: https://wordnet.princeton.edu/)

In [34]:
# replace contractions
# code borrowed from http://stackoverflow.com/questions/27845796/replacing-words-matching-regular-expressions-in-python
import re

replacement_patterns = [
(r'won\'t', 'will not'),
(r'can\'t', 'cannot'),
(r'i\'m', 'i am'),
(r'ain\'t', 'is not'),
(r'(\w+)\'ll', '\g<1> will'),
(r'(\w+)n\'t', '\g<1> not'),
(r'(\w+)\'ve', '\g<1> have'),
(r'(\w+)\'s', '\g<1> is'),
(r'(\w+)\'re', '\g<1> are'),
(r'(\w+)\'d', '\g<1> would')
]

class RegexpReplacer(object):
    def __init__(self, patterns=replacement_patterns):
        self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
    def replace(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            (s, count) = re.subn(pattern, repl, s)
        return s

In [41]:
sampletext = {'text':["I can't and won't ship this to Cleveland", "What's the name of this vessel?"]}
sample_df = pd.DataFrame(sampletext, columns = ['text'])
print(sample_df)



                                       text
0  I can't and won't ship this to Cleveland
1           What's the name of this vessel?


In [42]:
replacer = RegexpReplacer()
sample_df['text'] = sample_df['text'].map(lambda x: replacer.replace(x))

print(sample_df)

                                           text
0  I cannot and will not ship this to Cleveland
1              What is the name of this vessel?


### What changes might we want to make to our news stories?

Some basic changes:
* Plurals that we care about: year/years, yield/yields,  economic/economy, states/state, New York/NY
* Anything else? 

There are MANY ways to do this.  If you Google "dictionary replacement python", you will see lots of different options and competing code strings in StackOverflow responses.  

Here is one way to do it.  You should feel free to investigate others - especially if you have a large dictionary or a lot of text. 

In [43]:
# using dictionaries

news_dict = {'united states':'us', 'years':'year', 'states':'state', 'new york':'ny', 'mr. obama':'obama'}


def multiple_replace(dict, text): 

  """ Replace in 'text' all occurences of any key in the given
  dictionary by its corresponding value.  Returns the new tring.""" 
  text = str(text).lower()

  # Create a regular expression  from the dictionary keys
  regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())))

  # For each match, look-up corresponding value in dictionary
  return regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], text)



In [44]:
newsdf['cleantext'] = newsdf.text.apply(lambda x: multiple_replace(news_dict, x))
newsdf[0:1] 

date  \
0  2013-01-01   

                                                                                                                                                                description  \
0  Ending a climactic showdown in the final hours of the 112th Congress, the House sent to President Obama legislation to avert big income tax increases on most Americans.   

                                                         headline  \
0  Divided House Passes Tax Deal in End to Latest Fiscal Standoff   

                                                                              url  \
0  http://www.nytimes.com/2013/01/02/us/politics/house-takes-on-fiscal-cliff.html   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [45]:
# how does this preprocessing change the feature space?
cv_dm = cv.fit_transform(newsdf['cleantext'])
print(cv_dm.shape)
names = cv.get_feature_names()   
count = np.sum(cv_dm.toarray(), axis = 0)
count2 = count.tolist() 
count_df = pd.DataFrame(count2, index = names, columns = ['count']) 
count_df.sort_values(['count'], ascending = False)[0:29]  

(3848, 494)


,count
people,5547
new,5022
could,4620
last,4247
many,4211
first,4057
even,4029
state,3986
us,3869
city,3514


### Word analysis - comparing individual words to build classifications
Note:
##### Word used != topic
#### Remember: word choice is expression, topic is content

### Example:  
What is the topic for these words?
* base, run, out, homer

What is the topic for these words?
* bat, catch, home, fly

But words are important for classification:  assigning topic (two weeks) and assigning sentiment (this week)

## Sentiment dictionaries

Some packages do it for you:
http://text-processing.com/demo/sentiment/

Like other instances, how you do it depends on what you want to do.

Some options: 
* PL:
 * http://www.cs.cornell.edu/people/pabo/movie-review-data/
* AFINN: 
 * example - https://finnaarupnielsen.wordpress.com/2011/06/20/simplest-sentiment-analysis-in-python-with-af/
 * dictionary files: http://www2.imm.dtu.dk/pubdb/views/publication_details.php?id=6010
* HL: 
 * documentation: https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html
* Moby sentiment thesaurus:
 * http://papers.ssrn.com/sol3/papers.cfm?abstract_id=2191937
* Loughran and McDonald finance dictionaries:
 * http://www3.nd.edu/~mcdonald/Word_Lists.html
* General Inquirer - multi purpose and multidimensional
  * http://www.wjh.harvard.edu/~inquirer/homecat.htm

In [48]:
#some dictionaries assign a value 
afinn = {}
for line in open('C:\\Users\\Paul\\Desktop\\Rockhurst\\BIA 6304-Text Mining\\Week 4\\AFINN-111.txt'):
    tt = line.split('\t')
    afinn.update({tt[0]:int(tt[1])})


print(type(afinn), len(afinn))

for key, value in sorted(afinn.items())[0:10]:
    print(key + " => " + str(value))
print("~~~~~~~~~~~~")
for key, value in sorted(afinn.items())[2467:]:
    print(key + " => " + str(value))

<class 'dict'> 2477
abandon => -2
abandoned => -2
abandons => -2
abducted => -2
abduction => -2
abductions => -2
abhor => -3
abhorred => -3
abhorrent => -3
abhors => -3
~~~~~~~~~~~~
yeah => 1
yearning => 1
yeees => 2
yes => 1
youthful => 2
yucky => -2
yummy => 3
zealot => -2
zealots => -2
zealous => 2


In [50]:
#some sort into buckets

  
print("HL pos  size: " + str(len(HLpos)))
print(HLpos[0:10])
print("HL neg  size: " + str(len(HLneg)))
print(HLneg[0:10])

HL pos  size: 2006
['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']
HL neg  size: 4783
['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


### How can you use this? Need to construct a measure

Sum of words with values?

Count of positive over negative?


In [51]:
# here we are going for strictly the sum:  add up the positives and "subtract" the negatives
# you can return a number or a label

def afinn_sent(inputstring):
    
    sentcount =0
    for word in inputstring.split():  
        if word.rstrip('?:!.,;') in afinn:
            sentcount = sentcount + afinn[word.rstrip('?:!.,;')]
            
    
    if (sentcount < 0):
        sentiment = 'Negative'
    elif (sentcount >0):
        sentiment = 'Positive'
    else:
        sentiment = 'Neutral'
    
    return sentiment
    #return sentcount


In [52]:
import string
test_string = "5 years ago, I was shot in an attempt to stop me from speaking out for girls' education. Today, I attend my first lectures at Oxford."

print(test_string)
afinn_sent(test_string)


5 years ago, I was shot in an attempt to stop me from speaking out for girls' education. Today, I attend my first lectures at Oxford.


'Negative'

In [53]:
newsdf['afinn'] = newsdf.text.apply(lambda x: afinn_sent(x))
print(type(newsdf.text))

<class 'pandas.core.series.Series'>


In [54]:
newsdf.iloc[0:5][['text','afinn']]

text  \
0                                                                                                                                                                           WASHINGTON — Ending a climactic fiscal showdown in the final hours of the 112th Congress, the House late Tuesday passed and sent to   legislation to avert big income tax increases on most Americans and prevent large cuts in spending for the Pentagon and other government programs.         The measure, brought to the House floor less than 24 hours after its passage in the Senate, was approved 257 to 167, with 85 Republicans joining 172 Democrats in voting to allow income taxes to rise for the first time in two decades, in this case for the highest-earning Americans. Voting no were 151 Republicans and 16 Democrats.         The bill was expected to be signed quickly by Mr. Obama, who won re-election on a promise to increase taxes on the wealthy.         Mr. Obama strode into the White House briefing room shortly after the vote, less to hail the end of the fiscal crisis than to lay out a marker for the next one. “The one thing that I think, hopefully, the new year will focus on,” he said, “is seeing if we can put a package like this together with a little bit less drama, a little less brinkmanship, and not scare the heck out of folks quite as much.”         In approving the measure after days of legislative intrigue, Congress concluded its final and most pitched fight over fiscal policy, the culmination of two years of battles over taxes, the federal debt, spending and what to do to slow the growth in popular social programs like  .         The decision by Republican leaders to allow the vote came despite widespread scorn among House Republicans for the bill, passed overwhelmingly by the Senate in the early hours of New Year’s Day. They were unhappy that it did not include significant spending cuts in health and other social programs, which they say are essential to any long-term solution to the nation’s debt.         Democrats, while hardly placated by the compromise, celebrated Mr. Obama’s nominal victory in his final showdown with House Republicans in the 112th Congress, who began their term emboldened by scores of new, conservative members whose reach to the right ultimately tipped them over.         “The American people are the real winners tonight,” Representative Bill Pascrell Jr., Democrat of New Jersey, said on the House floor, “not anyone who navigates these halls.”         Not a single leader among House Republicans came to the floor to speak in favor of the bill, though Speaker  , who rarely takes part in roll calls, voted in favor. Representative Eric Cantor of Virginia, the majority leader, and Representative Kevin McCarthy of California, the No. 3 Republican, voted no. Representative Paul D. Ryan, the budget chairman who was the Republican vice-presidential candidate, supported the bill.         Despite the party divisions, many Republicans in their remarks characterized the measure, which allows taxes to go up on household income over $400,000 for individuals and $450,000 for couples but makes permanent tax cuts for income below that level, as a victory of sorts, even as so many of them declined to vote for it.         “After more than a decade of criticizing these tax cuts,” said Representative Dave Camp of Michigan, “Democrats are finally joining Republicans in making them permanent. Republicans and the American people are getting something really important, permanent tax relief.”         The dynamic with the House was a near replay of a fight at the end of 2011 over a   break extension. In that showdown, Senate Democrats and Republicans passed legislation, and while House Republicans fulminated, they were eventually forced to swallow it.         On Tuesday, as they got a detailed look at the Senate’s fiscal legislation, House Republicans ranging from Midwest pragmatists to  -blessed conservatives voiced serious reservations about the me

In [55]:
newsdf.iloc[3439:3440][['text','afinn']]

text  \
3439  The political party of the former cricket star Imran Khan on Wednesday identified a man it described as the C.I.A.'s top spy in Pakistan, in an escalation of Mr. Khan’s campaign to end American drone strikes in the country.         In a letter to the Pakistani police, Mr. Khan’s information secretary, Shireen Mazari, accused the C.I.A. director, John O. Brennan, along with a man identified as the agency’s Islamabad station chief, of “committing murder and waging war against Pakistan.”         In Washington, a C.I.A. spokesman declined to comment on the case.         Ms. Mazari demanded that the authorities prevent the station chief, whose identity has not yet been confirmed, from leaving the country so that he could face prosecution in a Pakistani court.         That seems unlikely, but the move is expected to infuriate American officials, who had to recall a previous C.I.A. station chief in 2010 after he was identified in the local news media, also in relation to a legal suit brought by anti-drone campaigners.         But while blame for that outing was placed on smoldering tensions between the C.I.A. and Pakistan’s Inter-Services Intelligence spy agency, this time it appears to be driven more by Mr. Khan’s increasingly confrontational stance against drone strikes.         In an appearance on a television talk show on Wednesday evening, Mr. Khan said he had named the station chief essentially to punish the C.I.A. for a deadly drone strike this month in the province his Pakistan Tehreek-e-Insaf party controls, Khyber-Pakhtunkhwa. Now, he said, it was up to the government of Prime Minister Nawaz Sharif to take the next step against the American spy agency.         He has vowed to block NATO supply lines into Afghanistan in retaliation for the Nov. 1 drone strike that  , Hakimullah Mehsud. On Saturday, his supporters moved to deliver on that promise by searching trucks and roughing up drivers as they passed through Khyber-Pakhtunkhwa on the way to Afghanistan.         In her letter on Wednesday, Ms. Mazari claimed that the station chief did not enjoy diplomatic immunity, and suggested that if interrogated by the police he might divulge the names of the pilots who fly the drones.         The high-profile attempt to obstruct C.I.A. operations in Pakistan was said to be a response to the Nov. 21 drone strike that   linked to the Haqqani network, a Taliban-affiliated militant group at the center of American security concerns in Afghanistan. The strike, which killed the Haqqanis’ spiritual leader and five others, occurred in the Hangu district of Khyber-Pakhtunkhwa, in a rare drone strike outside Pakistan’s tribal areas.         Mr. Khan has been a leading advocate of ceasing military action against the Pakistani Taliban, even though Khyber-Pakhtunkhwa has been the region hardest hit by Islamist violence this year, with hundreds killed in attacks. The Taliban also broke out many prisoners in an embarrassing and well-organized jailbreak in July.         Mr. Khan has used the drone issue to leverage his popularity against Mr. Sharif, who is his main electoral competitor in Punjab Province, and indeed has largely succeeded in framing the political debate on drones in recent years.         Some Sharif supporters criticized Mr. Khan for trying to score political points by outing the station chief. “This a thoughtless move,” said Siddiqul Farooq, a central leader of the governing Pakistan Muslim League party. “It is selfish and compromises the national interest.”         Since the escalation of the C.I.A.'s drone war in Pakistan in 2008, the Islamabad station has grown to become one of the spy agency’s largest outposts in the world. The agency’s expansion in Pakistan has been an irritant to America’s relations with Pakistan.         The influence of the C.I.A.'s Islamabad station chief has sometimes eclipsed even that of the American ambassador in Pakistan. A previous station chief clashed repeatedly in 2011 with Cameron Munt

In [56]:
# different dictionary
# different measure

def hl_sent(inputstring):

    poscount = 0
    negcount = 0
    
    for word in inputstring.split(): 
        if HLpos.count(word.rstrip('?:!.,;')):
            poscount +=1
        elif HLneg.count(word.rstrip('?:!.,;')):
            negcount +=1
     
    
    if poscount+negcount > 0:
        t = float((poscount - negcount)/(poscount+negcount))    
    else:
        t = 0
    
    
    if t > 0:
        tone = "Positive"
    elif t < 0:
        tone = "Negative"
    else:
        tone = "Neutral"
    
    return tone

In [57]:
print(test_string)
hl_sent(test_string)


5 years ago, I was shot in an attempt to stop me from speaking out for girls' education. Today, I attend my first lectures at Oxford.


'Neutral'

In [58]:
#takes a little longer - needs to check multiple lists
newsdf['hlsent'] = newsdf.text.apply(lambda x: hl_sent(x))

In [59]:
newsdf.iloc[0:10][['text','afinn', 'hlsent']]

text  \
0                                                                                                                                                                                                       WASHINGTON — Ending a climactic fiscal showdown in the final hours of the 112th Congress, the House late Tuesday passed and sent to   legislation to avert big income tax increases on most Americans and prevent large cuts in spending for the Pentagon and other government programs.         The measure, brought to the House floor less than 24 hours after its passage in the Senate, was approved 257 to 167, with 85 Republicans joining 172 Democrats in voting to allow income taxes to rise for the first time in two decades, in this case for the highest-earning Americans. Voting no were 151 Republicans and 16 Democrats.         The bill was expected to be signed quickly by Mr. Obama, who won re-election on a promise to increase taxes on the wealthy.         Mr. Obama strode into the White House briefing room shortly after the vote, less to hail the end of the fiscal crisis than to lay out a marker for the next one. “The one thing that I think, hopefully, the new year will focus on,” he said, “is seeing if we can put a package like this together with a little bit less drama, a little less brinkmanship, and not scare the heck out of folks quite as much.”         In approving the measure after days of legislative intrigue, Congress concluded its final and most pitched fight over fiscal policy, the culmination of two years of battles over taxes, the federal debt, spending and what to do to slow the growth in popular social programs like  .         The decision by Republican leaders to allow the vote came despite widespread scorn among House Republicans for the bill, passed overwhelmingly by the Senate in the early hours of New Year’s Day. They were unhappy that it did not include significant spending cuts in health and other social programs, which they say are essential to any long-term solution to the nation’s debt.         Democrats, while hardly placated by the compromise, celebrated Mr. Obama’s nominal victory in his final showdown with House Republicans in the 112th Congress, who began their term emboldened by scores of new, conservative members whose reach to the right ultimately tipped them over.         “The American people are the real winners tonight,” Representative Bill Pascrell Jr., Democrat of New Jersey, said on the House floor, “not anyone who navigates these halls.”         Not a single leader among House Republicans came to the floor to speak in favor of the bill, though Speaker  , who rarely takes part in roll calls, voted in favor. Representative Eric Cantor of Virginia, the majority leader, and Representative Kevin McCarthy of California, the No. 3 Republican, voted no. Representative Paul D. Ryan, the budget chairman who was the Republican vice-presidential candidate, supported the bill.         Despite the party divisions, many Republicans in their remarks characterized the measure, which allows taxes to go up on household income over $400,000 for individuals and $450,000 for couples but makes permanent tax cuts for income below that level, as a victory of sorts, even as so many of them declined to vote for it.         “After more than a decade of criticizing these tax cuts,” said Representative Dave Camp of Michigan, “Democrats are finally joining Republicans in making them permanent. Republicans and the American people are getting something really important, permanent tax relief.”         The dynamic with the House was a near replay of a fight at the end of 2011 over a   break extension. In that showdown, Senate Democrats and Republicans passed legislation, and while House Republicans fulminated, they were eventually forced to swallow it.         On Tuesday, as they got a detailed look at the Senate’s fiscal legislation, House Republicans ranging from Midwest pragmatists to  -blessed conservatives voiced serio

### One more dictionary: The General Inquirer dictionary
This one is interesting because it captures multiple dimensions and multiple definitions

In [ ]:
#new GI dictionary
import re

GI_frame = pd.read_csv(pathname +'inquirerbasic.csv',
                        usecols=[0,1,2,3,8,10,184,185], na_values = [" "])

GI_frame.fillna('', inplace=True)
print(GI_frame.shape)

# get rid of multiple counts
GI_frame['NewEntry'] = GI_frame['Entry'].str.extract('([A-Z]\w{0,})', expand = False)



In [ ]:
#what did we just do and what do we have now?
GI_frame.head(30)


In [ ]:
#let's create lists: pos, neg, strong, weak

GIlist = GI_frame['NewEntry'].tolist()
GIlist = list(map(lambda x: str(x).lower(), GIlist))
GIset = set(GIlist)
print("GI dictionary size: " + str(len(GIlist)) + " words of which " +str(len(GIset))+ " are unique.")


GIpos = GI_frame['NewEntry'][GI_frame['Positiv'].str.contains('Positiv')].tolist()
GIneg = GI_frame['NewEntry'][GI_frame['Negativ'].str.contains('Negativ')].tolist()
GIstrong = GI_frame['NewEntry'][GI_frame['Strong'].str.contains('Strong')].tolist()
GIweak = GI_frame['NewEntry'][GI_frame['Weak'].str.contains('Weak')].tolist()
GIpos = list(map(lambda x: str(x).lower(), GIpos))
GIneg = list(map(lambda x: str(x).lower(), GIneg))
GIstrong = list(map(lambda x: str(x).lower(), GIstrong))
GIweak = list(map(lambda x: str(x).lower(), GIweak))

print("Positive words: " + str(len(GIpos)))
print("Negative words: " + str(len(GIneg)))
print("Strong words: "+ str(len(GIstrong)))
print("Weak words: " + str(len(GIweak)))



In [ ]:
#what do we have? some overlap based on usage 
# Decisions will need to be made on how to deal with these

print(set(GIpos).intersection(set(GIneg)))
print("~~~~~~")
print (set(GIstrong).intersection(set(GIweak)))

In [ ]:
#Let's take the most common use based on info in the "defined" column
# We could also use the POS here

#define function to pull out percent
def get_digits(text):
    temp_num = ''.join(list(filter(str.isdigit, text)))
    if temp_num == '':
        temp_num = 100
    return temp_num

#create new column for digits - not strictly necessary since it's not relevant for all entries but fastest way
GI_frame['Percent'] = GI_frame['Defined'].map(lambda x: get_digits(x))

print(GI_frame.shape)

In [ ]:
print(GI_frame.head(30))

In [ ]:
#pull out only the versions of terms used more than 50% of the time
GI_frame.Percent.astype(int) > 50
GI_frame = GI_frame[GI_frame.Percent.astype(int) > 50]
print(GI_frame.shape)

In [ ]:
GIlist = GI_frame['NewEntry'].tolist()
GIlist = list(map(lambda x: str(x).lower(), GIlist))
GIset = set(GIlist)
print("GI dictionary size: " + str(len(GIlist)) + " words of which " +str(len(GIset))+ " are unique.")


GIpos = GI_frame['NewEntry'][GI_frame['Positiv'].str.contains('Positiv')].tolist()
GIneg = GI_frame['NewEntry'][GI_frame['Negativ'].str.contains('Negativ')].tolist()
GIstrong = GI_frame['NewEntry'][GI_frame['Strong'].str.contains('Strong')].tolist()
GIweak = GI_frame['NewEntry'][GI_frame['Weak'].str.contains('Weak')].tolist()
GIpos = list(map(lambda x: str(x).lower(), GIpos))
GIneg = list(map(lambda x: str(x).lower(), GIneg))
GIstrong = list(map(lambda x: str(x).lower(), GIstrong))
GIweak = list(map(lambda x: str(x).lower(), GIweak))

print("Positive words: " + str(len(GIpos)))
print("Negative words: " + str(len(GIneg)))
print("Strong words: "+ str(len(GIstrong)))
print("Weak words: " + str(len(GIweak)))



In [ ]:
# We fixed the orientation duplication but didn't deal with the strength clashes.  

print(set(GIpos).intersection(set(GIneg)))
print("~~~~~~")
print (set(GIstrong).intersection(set(GIweak)))

In [ ]:
#let's create index


def gi_sent(inputstring, show = None):

    poscount = 0
    negcount = 0
    i = 0


    for word in inputstring.split():
        if i > 0:
            prev = inputstring.split().pop(i-1)
        else:
            prev =""

        #create scalar for strong and weak words.  Strong words double, weak words add half
        if GIstrong.count(word):
            scale = 2
            if show != None:
                print("Strong: " + word) 
        elif GIweak.count(word):
            if show != None: 
                print("Weak: " + word)
            scale = 0.5
        else:
            scale = 1
            
        if GIpos.count(word):
            if show != None:
                print("Postive: " + word ) 
            poscount +=1*scale
        elif GIneg.count(word):
            if show != None:
                print("Negative: " + word )
            negcount +=1*scale
            
        i+=1
    
    if poscount+negcount > 0:
        t = float((poscount - negcount)/(poscount+negcount))
        
    else:
        t = 0
    
    
    if t > 0:
        tone = "Positive"
    elif t < 0:
        tone = "Negative"
    else:
        tone = "Neutral"
    
    return tone

In [ ]:
print(test_string)
gi_sent(test_string)


In [ ]:
#takes a little longer - needs to check even more lists
newsdf['gisent'] = newsdf.text.apply(lambda x: gi_sent(x))

In [ ]:
newsdf.iloc[0:10][['text','afinn', 'hlsent', 'gisent']]

### How do you think stemming affects the analysis? 
http://sentiment.christopherpotts.net/stemming.html

### A different application/definition of "tone"

http://www.fastcompany.com/3048761/tech-forecast/watch-your-tone-now-ibms-watson-can-detect-attitude-in-your-writing

### Classification isn't just by "sentiment"

https://www.nytimes.com/2017/01/16/upshot/job-disconnect-male-applicants-feminine-language.html?_r=0

http://www.dailymail.co.uk/sciencetech/article-3463607/The-simple-trick-reveal-conservative-liberal-Study-discovers-words-away-political-beliefs.html


## In practice, sentiment analyzers will create a model from a vast corpus and then apply that to a new set of text instead of scoring each new document directly

http://streamhacker.com/2010/05/10/text-classification-sentiment-analysis-naive-bayes-classifier/

http://www.laurentluce.com/posts/twitter-sentiment-analysis-using-python-and-nltk/

https://marcobonzanini.com/2015/01/19/sentiment-analysis-with-python-and-scikit-learn/ This one cheats IMHO because the dataset is already tagged...

In [ ]:
#more complicated sentiment calculations
#create all the dictionaries just once

#amplification and negation words from qdap
negate = ["aint", "arent","cant", "couldnt" , "didnt" , "doesnt" ,"dont" ,"hasnt" , "isnt" ,"mightnt" , "mustnt" ,"neither" ,"never", "no" ,"nobody" , "nor", "not" , "shant", "shouldnt", "wasnt" , "werent" ,"wont", "wouldnt"]
amplify = ["acute" ,"acutely", "certain", "certainly" ,"colossal", "colossally","deep" , "deeply" , "definite","definitely" ,"enormous","enormously" , "extreme", "extremely" ,"great","greatly" ,"heavily", "heavy", "high","highly" ,"huge","hugely" , "immense", "immensely" ,"incalculable" ,"incalculably","massive", "massively", "more","particular" ,"particularly","purpose", "purposely", "quite" ,"real" ,"really","serious", "seriously", "severe","severely" ,"significant" ,"significantly","sure","surely" , "true" ,"truly" ,"vast" , "vastly" , "very"]




In [ ]:
def afinn_sent2(inputstring):
    
    sentcount =0
    i=0
    

    for word in inputstring.split():
        prev = inputstring.split().pop(i-1)

        if word in afinn:
            if (prev == 'no'):
                sentcount = sentcount - afinn[word] - afinn[prev]
            elif (prev == 'not'):
                sentcount = sentcount - afinn[word]
            else:
                sentcount = sentcount + afinn[word]
            i+=1
    
    if (sentcount < 0):
        sentiment = 'Negative'
    elif (sentcount >0):
        sentiment = 'Positive'
    else:
        sentiment = 'Neutral'
    
    
    return sentiment

def hl_sent2(inputstring):

    poscount = 0
    negcount = 0
    i = 0


    for word in inputstring.split():
        if i > 0:
            prev = inputstring.split().pop(i-1)
        else:
            prev =""

        if HLpos.count(word):
            if negate.count(prev):
                negcount += 1
            elif amplify.count(prev):
                poscount +=2
            else: 
                poscount +=1
        elif HLneg.count(word):
            if negate.count(prev):
                poscount += 1
            elif amplify.count(prev):
                negcount +=2
            else:
                negcount +=1
        i+=1
    
    if poscount+negcount > 0:
        t = float((poscount - negcount)/(poscount+negcount))
        
    else:
        t = 0
    
    
    if t > 0:
        tone = "Positive"
    elif t < 0:
        tone = "Negative"
    else:
        tone = "Neutral"
    
    return tone

#let's create lists: pos, neg, strong, weak


def gi_sent2(inputstring):

    poscount = 0
    negcount = 0
    i = 0


    for word in inputstring.split():
        if i > 0:
            prev = inputstring.split().pop(i-1)
        else:
            prev =""

        #create scalar for strong and weak words.  Strong words double, weak words add half
        if GIstrong.count(word):
            scale = 2
        elif GIweak.count(word):
            scale = 0.5
        else:
            scale = 1
            
        if GIpos.count(word):
            if negate.count(prev):
                negcount += 1*scale
            elif amplify.count(prev):
                poscount +=2*scale
            else: 
                poscount +=1*scale
        elif GIneg.count(word):
            if negate.count(prev):
                poscount += 1*scale
            elif amplify.count(prev):
                negcount +=2*scale
            else:
                negcount +=1*scale
            
        i+=1
    
    if poscount+negcount > 0:
        t = float((poscount - negcount)/(poscount+negcount))
        
    else:
        t = 0
    
    
    if t > 0:
        tone = "Positive"
    elif t < 0:
        tone = "Negative"
    else:
        tone = "Neutral"
    

    #return (negcount, poscount)
    #return tone
    return tone

### Does this affect our labels?

In [ ]:
test_string = "This class not stinks.  I really not hate it.  Just kidding.  it's the great, the best even."

In [ ]:
print(test_string)
afinn_sent2(test_string)

In [ ]:
hl_sent2(test_string)

In [ ]:
gi_sent2(test_string)

In [ ]:
newsdf['afinn2'] = newsdf.text.apply(lambda x: afinn_sent2(x))

In [ ]:
newsdf.iloc[269:270][['text','afinn',  'afinn2', 'hlsent','gisent']]

### So does this make a big difference?  How many places does this change things?

In [ ]:
diff = np.where((newsdf['afinn'] != newsdf['afinn2']))

In [ ]:
print(type(diff))
print(diff)

In [ ]:
# So how many articles flipped labels?
print(len(diff[0]))


### Main take aways:
* Sets and lists are handy tools for object manipulation but make sure you understand how to use each of them
* Custom replacement dictionaries are very useful for being more selective in your treatment of words and are especially helpful where business-specific language needs to be considered
* Sentiment analysis requires some determination of semantic orientation for words which may be context dependent.  
* What dictionary to use and how to calculate your index are more choices that the data scientist needs to make (on top of stop word lists, stemming/lemmatization, vectorizer options, parameter settings, etc.)